In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
df = pd.read_csv('/content/Campaign-Data.csv')
df.columns

In [ ]:
df['Calendardate'] = pd.to_datetime(df['Calendardate'])
df['Month'] = df['Calendardate'].dt.month
df['Year'] = df['Calendardate'].dt.year

In [ ]:
df['Client Type'].value_counts(normalize = True)

In [ ]:
pd.crosstab(df['Number of Competition'] , df['Client Type'] , margins = True , normalize = 'columns')

In [ ]:
df.groupby('Number of Competition').mean()

In [ ]:
df.groupby('Client Type').mean()

In [ ]:
df.corr()[['Amount Collected']]

In [27]:
cm = sns.light_palette("green" , as_cmap = True)
corr_analysis = pd.DataFrame(df[['Amount Collected','Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
       'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5']].corr()['Amount Collected']).reset_index()
corr_analysis.columns = ['Variables' , 'Correlation']
corr_analysis = corr_analysis[corr_analysis['Variables'] != 'Amount Collected']
corr_analysis = corr_analysis.sort_values('Correlation' , ascending = False)
corr_analysis.style.background_gradient(cmap = cm).set_precision(2)

<ipython-input-27-708c4fde1d15>:8: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr_analysis.style.background_gradient(cmap = cm).set_precision(2)


,Variables,Correlation
5,Sales Contact 2,0.55
2,Campaign (Flyer),0.44
6,Sales Contact 3,0.36
4,Sales Contact 1,0.28
1,Campaign (Email),0.25
7,Sales Contact 4,0.24
8,Sales Contact 5,0.10
3,Campaign (Phone),0.03


In [34]:
cm = sns.light_palette("green" , as_cmap = True)
corr_analysis = pd.DataFrame(df.groupby('Client Type')[['Amount Collected','Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
       'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5']].corr()['Amount Collected']).reset_index()
corr_analysis = corr_analysis.sort_values(['Client Type' , 'Amount Collected'] , ascending = False)
corr_analysis.columns = ['Acc Type','Variables' , 'Correlation']
corr_analysis = corr_analysis[corr_analysis['Variables'] != 'Amount Collected'].reset_index(drop = True)
corr_analysis.style.background_gradient(cmap = cm).set_precision(2)

<ipython-input-34-989a458e4d20>:8: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr_analysis.style.background_gradient(cmap = cm).set_precision(2)


,Acc Type,Variables,Correlation
0,Small Facility,Sales Contact 2,0.22
1,Small Facility,Sales Contact 3,0.07
2,Small Facility,Campaign (Email),0.06
3,Small Facility,Campaign (Flyer),0.04
4,Small Facility,Sales Contact 4,0.02
5,Small Facility,Sales Contact 5,0.00
6,Small Facility,Sales Contact 1,-0.02
7,Small Facility,Campaign (Phone),nan
8,Private Facility,Sales Contact 2,0.57
9,Private Facility,Campaign (Flyer),0.28


In [41]:
df.columns=[mystring.replace(" ", "_") for mystring in df.columns]
df.columns=[mystring.replace("(", "") for mystring in df.columns]
df.columns=[mystring.replace(")", "") for mystring in df.columns]
results = smf.ols('Amount_Collected ~ Campaign_Email+Campaign_Flyer+Campaign_Phone+\
       Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3+Sales_Contact_4 + Sales_Contact_5',data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       Amount_Collected   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.478
Method:                 Least Squares   F-statistic:                     342.1
Date:                Sat, 04 Nov 2023   Prob (F-statistic):               0.00
Time:                        14:47:23   Log-Likelihood:                -54512.
No. Observations:                2976   AIC:                         1.090e+05
Df Residuals:                    2967   BIC:                         1.091e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        1.481e+06   5.12e+05     

In [47]:
data = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0]
data=data.reset_index()
data=data[data['P>|t|']<0.05][['index','coef']]
data

,index,coef
5,Sales_Contact_2,6.6223


In [48]:
consolidated_summary=pd.DataFrame()
for acctype in list(set(list(df['Client_Type']))):
    print(acctype)
    temp_data=df[df['Client_Type']==acctype].copy()
    results = smf.ols('Amount_Collected ~ Campaign_Email+Campaign_Flyer+Campaign_Phone+\
       Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3+Sales_Contact_4 + Sales_Contact_5', data=temp_data).fit()
    data = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
    data=data[data['P>|t|']<0.05][['index','coef']]
    data.columns=['Variable','Coefficent (Impact)']
    data['Account Type']=acctype
    data=data.sort_values('Coefficent (Impact)',ascending=False)
    data=data[data['Variable']!='Intercept']
    consolidated_summary=consolidated_summary.append(data)
    print(results.summary())


Small Facility 
                            OLS Regression Results                            
Dep. Variable:       Amount_Collected   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     6.829
Date:                Sat, 04 Nov 2023   Prob (F-statistic):           6.63e-08
Time:                        15:04:32   Log-Likelihood:                -14125.
No. Observations:                 840   AIC:                         2.827e+04
Df Residuals:                     832   BIC:                         2.830e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        8.789e+05

<ipython-input-48-cb9e5822125b>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  consolidated_summary=consolidated_summary.append(data)
<ipython-input-48-cb9e5822125b>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  consolidated_summary=consolidated_summary.append(data)
<ipython-input-48-cb9e5822125b>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  consolidated_summary=consolidated_summary.append(data)
<ipython-input-48-cb9e5822125b>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  consolidated_summary=consolidated_summary.append(data)


In [49]:
consolidated_summary

,Variable,Coefficent (Impact),Account Type
5,Sales_Contact_2,0.810100,Small Facility
3,Campaign_Phone,-0.000003,Small Facility
2,Campaign_Flyer,4.105900,Medium Facility
5,Sales_Contact_2,3.577800,Medium Facility
4,Sales_Contact_1,3.136500,Medium Facility
6,Sales_Contact_3,2.117400,Medium Facility
4,Sales_Contact_1,11.673100,Large Facility
7,Sales_Contact_4,10.614500,Large Facility
5,Sales_Contact_2,4.003100,Large Facility
2,Campaign_Flyer,2.720400,Large Facility
